# Exercise: Complete Modeling Workflow

**Download: ``http://shorturl.at/crOS2/Lecture_12``**

File obsdata.csv contains four columns, each with 20 points. The columns are: time, A, B, C. 
1. Evaluate each of the following models using cross validation to find which provides the best fit.

    Model 1:

        A -> B

        B -> C


    Model 2:

         A -> B

         A -> C

    Model 3:

         A + B -> C

    All kinetics are mass action. A is starts at 5M, and all other concentrations are 0. In Model 3, A is 2.5 and B is 2.5.
    Kinetics constants are in [0, 10]
    
    a. Write the full Antimony models for each of the above.
    
    b. What simulation time should you use? How many points?
    
    c. Do cross validation on each. Which has the largest $R^2$ across folds?
    
    d. Did some folds in cross validation produce much better fits than others? Why?

2. Construct a 95% confidence interval for the model with the best fit.

3. The "observational data" were generated by a model that contains a chemical species D that is not present in the other models. How might we detect that there is a missing chemical species?

Hint: You can read data in a CSV file into a numpy array using ``np.genfromtxt("obsdata.csv", delimiter=",", skip_header=True)``.

## Checkpoints
1. Antimony models. Insert kinetics and constant initializations.
1. Plot observational data, identifying time periods over which fitting is done.
1. Cross validation results
   1. Model 1
   1. Model 2
   1. Model 3
1. Choice for "best model"
1. Parameter estimates and evaluation.